In [1]:
# Dependencies

import requests  
#The requests module allows us to send HTTP requests using Python

from bs4 import BeautifulSoup  
#This line is used to import the Beautiful Soup library into the application.

# import Pandas Library
import pandas as pd  

# import RE Library
import re 
#Regular expression operations Library the functions in this module let us check 
#if a particular string matches a given regular expression

#Using Splinter to automate browser actions
from splinter import Browser  

In [2]:
#choosing chrom driver 
!which chromedriver

/usr/local/bin/chromedriver


In [8]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [9]:
query = '\"Movie\" and \"Review\" '.lower()  
url = f'https://www.bing.com/news/search?q={query}&form=QBNH'

In [10]:
# visiting the Url in the our browser 
browser.visit(url)

In [14]:
html = browser.html

In [15]:
#Creat a Beautiful Soup object and the object returned is assigned to the soup variable.
soup = BeautifulSoup(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [16]:
reviews = soup.find_all('div', class_='t_s', limit = 50)
print(f'Number of scraped news: {len(reviews)}') #check out if we have 20

Number of scraped news: 50


In [17]:
titles = [n.find('a',class_ = 'title').text for  n  in reviews]   #.text :Extract the text of the tittle
titles

['‘Jakob’s Wife’ Review: A Cheap and Cheerless Vampire Movie Knock-Off',
 '‘City of Lies’ Review: Johnny Depp Solves the Notorious B.I.G.’s Murder in a Misshapen Detective Thriller',
 '‘Deadly Illusions’ Review: Kristin Davis Hires the Wrong Nanny in Silly, Sexy Thriller',
 'Snyder Cut Justice League review: Still a mess, now a million years long',
 'The Courier movie review: An intimate portrayal of spies and secrets during the Cold War',
 '‘Falcon And The Winter Soldier’ Review: Thanks, I Hate Him',
 '‘Wojnarowicz’ Review: A Queer Biography as Bracing and Rebellious as the Artist Himself',
 'Mumbai Saga Public Review: Interesting After The Interval And Climax is Worthy of Goosebumps, Say The Audience',
 "Movie Review | 'The Courier' is deliberate, intelligent, and moves quickly",
 'FilmWeek: ‘The Courier,’ ‘Zack Snyder’s Justice League,’ ‘City Of Lies’ And More',
 'The Feast (SXSW 2021 Movie Review)',
 "Review: 'Zack Snyder's Justice League' has arrived. It's been a long wait — and s

In [69]:
reviews_df = pd.DataFrame({'title': titles} )
reviews_df.head()

,title
0,‘Jakob’s Wife’ Review: A Cheap and Cheerless V...
1,‘City of Lies’ Review: Johnny Depp Solves the ...
2,‘Deadly Illusions’ Review: Kristin Davis Hires...
3,Snyder Cut Justice League review: Still a mess...
4,The Courier movie review: An intimate portraya...


In [61]:
import numpy as np
import pandas as pd
from afinn import Afinn

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import string
import re 


In [70]:
from nltk.corpus import stopwords 

stop_words = set(stopwords.words('english') + stopwords.words('spanish'))

lemmer = WordNetLemmatizer()

def preprocess(x):
    # Lower case
    x = x.lower()
    
    # Remove punctuation
    x = re.sub(r'[^\w\s\-]', ' ', x)
   #x = re.sub(r'[^\P{P}\-]+', ' ', x)
    
    
    # Remove numbers
    x = re.sub(r'\d+', '', x)
    
    # Remove stopwords and lemmatize
    x = [lemmer.lemmatize(w) for w in x.split() if w not in stop_words]
    return ' '.join(x) 

reviews_df['title_clean'] = reviews_df['title'].apply(preprocess)

reviews_df.head()

,title,title_clean
0,‘Jakob’s Wife’ Review: A Cheap and Cheerless V...,jakob wife review cheap cheerless vampire movi...
1,‘City of Lies’ Review: Johnny Depp Solves the ...,city lie review johnny depp solves notorious b...
2,‘Deadly Illusions’ Review: Kristin Davis Hires...,deadly illusion review kristin davis hire wron...
3,Snyder Cut Justice League review: Still a mess...,snyder cut justice league review still mess mi...
4,The Courier movie review: An intimate portraya...,courier movie review intimate portrayal spy se...


In [71]:
# Sentiment analysis with AFINN
afinn = Afinn(emoticons=True)
afinn_scores = [afinn.score(text) for text in reviews_df.title_clean]
reviews_df['afinn_sentiment'] = afinn_scores

reviews_df[[ 'afinn_sentiment']].head(20)

,afinn_sentiment
0,-2.0
1,-3.0
2,-3.0
3,-1.0
4,-2.0
5,-1.0
6,0.0
7,4.0
8,2.0
9,2.0


In [22]:
 from nltk.sentiment import SentimentIntensityAnalyzer

In [72]:
# Sentiment analysis using nltk_SentimentIntensityAnalyzer

Sent = SentimentIntensityAnalyzer()
SIA = [Sent.polarity_scores(text).get('compound') for text in reviews_df.title_clean]
reviews_df['nltk_sentiment'] = SIA 

reviews_df[[ 'afinn_sentiment', 'nltk_sentiment']]

,afinn_sentiment,nltk_sentiment
0,-2.0,-0.4019
1,-3.0,-0.7269
2,-3.0,0.2023
3,-1.0,-0.0516
4,-2.0,-0.5994
5,-1.0,-0.2023
6,0.0,-0.2960
7,4.0,0.6808
8,2.0,0.4588
9,2.0,0.5267


In [67]:
from textblob import TextBlob

In [73]:

SIA = [TextBlob(text).sentiment.polarity for text in reviews_df.title_clean]
reviews_df['TextBlob_sentiment'] = SIA 
reviews_df[['title', 'afinn_sentiment','nltk_sentiment','TextBlob_sentiment']]

,title,afinn_sentiment,nltk_sentiment,TextBlob_sentiment
0,‘Jakob’s Wife’ Review: A Cheap and Cheerless V...,-2.0,-0.4019,0.400000
1,‘City of Lies’ Review: Johnny Depp Solves the ...,-3.0,-0.7269,0.000000
2,‘Deadly Illusions’ Review: Kristin Davis Hires...,-3.0,0.2023,-0.175000
3,Snyder Cut Justice League review: Still a mess...,-1.0,-0.0516,-0.112500
4,The Courier movie review: An intimate portraya...,-2.0,-0.5994,-0.266667
5,‘Falcon And The Winter Soldier’ Review: Thanks...,-1.0,-0.2023,-0.300000
6,‘Wojnarowicz’ Review: A Queer Biography as Bra...,0.0,-0.2960,0.000000
7,Mumbai Saga Public Review: Interesting After T...,4.0,0.6808,0.277778
8,"Movie Review | 'The Courier' is deliberate, in...",2.0,0.4588,0.566667
9,"FilmWeek: ‘The Courier,’ ‘Zack Snyder’s Justic...",2.0,0.5267,0.000000


In [27]:
from transformers import pipeline
classify = pipeline('sentiment-analysis')

In [75]:
def get_score(text):
    sent = classify(text)
    score = sent[0].get('score')
    return score

def get_label (text):
    sent = classify(text)
    label = sent[0].get('label')
    return label

In [77]:
transf_sc = [get_score(text) for text in reviews_df.title]
reviews_df['transformers_score'] = transf_sc

transf_l = [get_label(text) for text in reviews_df.title]
reviews_df['transformers_label'] = transf_l
reviews_df[['title', 'afinn_sentiment','nltk_sentiment','TextBlob_sentiment','transformers_score','transformers_label']]


,title,afinn_sentiment,nltk_sentiment,TextBlob_sentiment,transformers_score,transformers_label
0,‘Jakob’s Wife’ Review: A Cheap and Cheerless V...,-2.0,-0.4019,0.400000,0.999514,NEGATIVE
1,‘City of Lies’ Review: Johnny Depp Solves the ...,-3.0,-0.7269,0.000000,0.958238,NEGATIVE
2,‘Deadly Illusions’ Review: Kristin Davis Hires...,-3.0,0.2023,-0.175000,0.992938,POSITIVE
3,Snyder Cut Justice League review: Still a mess...,-1.0,-0.0516,-0.112500,0.999426,NEGATIVE
4,The Courier movie review: An intimate portraya...,-2.0,-0.5994,-0.266667,0.999396,POSITIVE
5,‘Falcon And The Winter Soldier’ Review: Thanks...,-1.0,-0.2023,-0.300000,0.920812,NEGATIVE
6,‘Wojnarowicz’ Review: A Queer Biography as Bra...,0.0,-0.2960,0.000000,0.996386,POSITIVE
7,Mumbai Saga Public Review: Interesting After T...,4.0,0.6808,0.277778,0.999556,POSITIVE
8,"Movie Review | 'The Courier' is deliberate, in...",2.0,0.4588,0.566667,0.999755,POSITIVE
9,"FilmWeek: ‘The Courier,’ ‘Zack Snyder’s Justic...",2.0,0.5267,0.000000,0.774785,POSITIVE


In [79]:
reviews_df[['nltk_sentiment','TextBlob_sentiment','transformers_score']] = reviews_df[['nltk_sentiment','TextBlob_sentiment','transformers_score']].apply(lambda x: pd.Series.round(x,2)) 
reviews_df.head()

,title,title_clean,afinn_sentiment,nltk_sentiment,TextBlob_sentiment,transformers_score,transformers_label
0,‘Jakob’s Wife’ Review: A Cheap and Cheerless V...,jakob wife review cheap cheerless vampire movi...,-2.0,-0.40,0.40,1.00,NEGATIVE
1,‘City of Lies’ Review: Johnny Depp Solves the ...,city lie review johnny depp solves notorious b...,-3.0,-0.73,0.00,0.96,NEGATIVE
2,‘Deadly Illusions’ Review: Kristin Davis Hires...,deadly illusion review kristin davis hire wron...,-3.0,0.20,-0.18,0.99,POSITIVE
3,Snyder Cut Justice League review: Still a mess...,snyder cut justice league review still mess mi...,-1.0,-0.05,-0.11,1.00,NEGATIVE
4,The Courier movie review: An intimate portraya...,courier movie review intimate portrayal spy se...,-2.0,-0.60,-0.27,1.00,POSITIVE


In [39]:
reviews_20 = reviews_df.head(20)

In [80]:
reviews_df.to_excel('reviews_50.xlsx', index = False)

In [41]:
reviews_20.to_excel('reviews20.xlsx', index = False)

In [42]:
reviews_20.to_csv('reviews20.csv', index = False)

In [43]:

reviews_df.to_csv('reviews_50.csv', index = False)